In [38]:
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [39]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

Create training and test set

In [40]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())

for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

Predefine hyperparameters

In [41]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

Tokenize 

In [42]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

Create model.
Most important part here is Embedding layer. See Keras doc:

https://keras.io/layers/embeddings/

Keras offers an Embedding layer that can be used for neural networks on text data.

It requires that the input data be integer encoded, so that each word is represented by a unique integer. 
This data preparation step can be performed using the Tokenizer API also provided with Keras.

The Embedding layer is initialized with random weights and will learn an embedding for all of the words in the training dataset.

Source:

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

and additional source:

https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work

In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Compile the model

In [44]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_16 (Dense)             (None, 24)                1560      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 25        
Total params: 174,129
Trainable params: 174,129
Non-trainable params: 0
_________________________________________________________________


Train the model

In [45]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 58s 2ms/sample - loss: 0.4577 - accuracy: 0.7716 - val_loss: 0.3556 - val_accuracy: 0.8415
Epoch 2/10
25000/25000 [==============================] - 53s 2ms/sample - loss: 0.2863 - accuracy: 0.8841 - val_loss: 0.3530 - val_accuracy: 0.8450
Epoch 3/10
25000/25000 [==============================] - 54s 2ms/sample - loss: 0.2267 - accuracy: 0.9147 - val_loss: 0.4162 - val_accuracy: 0.8350
Epoch 4/10
25000/25000 [==============================] - 53s 2ms/sample - loss: 0.1718 - accuracy: 0.9363 - val_loss: 0.4790 - val_accuracy: 0.8334
Epoch 5/10
25000/25000 [==============================] - 56s 2ms/sample - loss: 0.1248 - accuracy: 0.9555 - val_loss: 0.6132 - val_accuracy: 0.8222
Epoch 6/10
25000/25000 [==============================] - 56s 2ms/sample - loss: 0.0855 - accuracy: 0.9704 - val_loss: 0.6700 - val_accuracy: 0.8242
Epoch 7/10
25000/25000 [==============================] 

Show the shape of model

In [46]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


Decode token list

In [47]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b perhaps because i was so young innocent and <OOV> when i saw it this movie was the cause of many <OOV> nights for me i haven't seen it since i was in seventh grade at a <OOV> school so i am not sure what effect it would have on me now however i will say that it left an impression on me and most of my friends it did serve its purpose at least until we were old enough and <OOV> enough to analyze and create our own opinions i was particularly terrified of what the newly converted post rapture christians had to endure when not receiving the mark of the beast i don't want to spoil the movie
b"Perhaps because I was so young, innocent and BRAINWASHED when I saw it, this movie was the cause of many sleepless nights for me. I haven't seen it since I was in seventh grade at a Presbyterian school, so I am not sure what effect it would have on me now. However, I will say that it left an impression on me... and most of my friends. It did serve its purpose, at least until we were old enough and kn

Write vectors from metadata.
It can be later used in https://projector.tensorflow.org/

In [48]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")

# Write each value of embeddings
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [49]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [50]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1429], [969], [4], [1537], [1537], [4749], [], [790], [2012], [11], [2920], [2188], [], [790], [2012], [11], [579], [], [11], [579], [], [4], [1783], [4], [4512], [11], [2920], [1277], [], [], [2012], [1003], [2920], [969], [579], [790], []]
